# Worksheet 07

Name:  Zheyu Yang
UID: U41174832

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.297132316296735, 4.564779964342392, 5.758569338192415, 4.442725423309262, 5.732727054026803, 5.31940711770437, 4.567693336732242, 4.976302980759255, 4.763764303650514, 4.8991956950146935]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
c2 = np.random.normal(8, 1 ,10).tolist()
print(c2)

[8.150533539740557, 8.311889916295524, 7.018958027607782, 8.860217994505856, 7.51023453370241, 8.217354951892062, 8.62654112690827, 7.475098066322631, 8.792484932751748, 6.974940885734358]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[6.974940885734358, 8.792484932751748, 4.8991956950146935, 4.763764303650514, 4.976302980759255, 7.475098066322631, 4.567693336732242, 8.62654112690827, 8.217354951892062, 5.31940711770437]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.8991956950146935, 4.763764303650514, 4.976302980759255, 4.567693336732242, 5.31940711770437]
[6.974940885734358, 8.792484932751748, 7.475098066322631, 8.62654112690827, 8.217354951892062]
P(C_1) = 0.5,  P(C_2) = 0.5
mean_1 = 4.905272686772215,  mean_2 = 8.017283992721813
var_1 = 0.0621147998886669,  var_2 = 0.4785207746306587


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[1] / prob_x )
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  6.974940885734358
probability of observing that point if it came from cluster 0 =  5.3121403296778224e-241
probability of observing that point if it came from cluster 1 =  0.07774716300628617
point =  8.792484932751748
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.22445642484419295
point =  4.8991956950146935
probability of observing that point if it came from cluster 0 =  6.39199632414613
probability of observing that point if it came from cluster 1 =  5.023872234376289e-10
point =  4.763764303650514
probability of observing that point if it came from cluster 0 =  0.4794056542184691
probability of observing that point if it came from cluster 1 =  7.633478028933959e-11
point =  4.976302980759255
probability of observing that point if it came from cluster 0 =  3.340094644674226
probability of observing that point if it came from cluster 1 =  1.4171053270170279e-09
point =  7.47509806632263

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x)]
var = [sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x), sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x)]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.4013503260609375,  P(C_2) = 0.5986496739390625
mean_1 = 4.803481078481586,  mean_2 = 7.572708781149938
var_1 = 0.024865219823867597,  var_2 = 1.4014296005545233


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = []
prob_c1_x = []

for p in data:
    pdf_i = [norm.pdf(p, mean[j], var[j]) for j in range(k)]
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()

6.974940885734358
Probability of coming from C_1 = 0.0
Probability of coming from C_2 = 1.0

8.792484932751748
Probability of coming from C_1 = 0.0
Probability of coming from C_2 = 1.0

4.8991956950146935
Probability of coming from C_1 = 0.12378180327449577
Probability of coming from C_2 = 0.8762181967255042

4.763764303650514
Probability of coming from C_1 = 0.9874446638046273
Probability of coming from C_2 = 0.012555336195372715

4.976302980759255
Probability of coming from C_1 = 6.80569556781465e-09
Probability of coming from C_2 = 0.9999999931943044

7.475098066322631
Probability of coming from C_1 = 0.0
Probability of coming from C_2 = 1.0

4.567693336732242
Probability of coming from C_1 = 1.1214553267490635e-17
Probability of coming from C_2 = 1.0

8.62654112690827
Probability of coming from C_1 = 0.0
Probability of coming from C_2 = 1.0

8.217354951892062
Probability of coming from C_1 = 0.0
Probability of coming from C_2 = 1.0

5.31940711770437
Probability of coming from C_1 =

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [10]:
labels = [0 if prob_c0_x[i] > prob_c1_x[i] else 1 for i in range(len(data))]
print(labels)

[1, 1, 1, 0, 1, 1, 1, 1, 1, 1]


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.